In [184]:
from bs4 import BeautifulSoup
import requests
import time
import csv
import os
import re
import pickle

In [239]:
class Job:
    def __init__(self, URI, description , summary):
        self.URI = URI
        self.description = description
        self.summary = summary
    def __iter__(self):
        return self
    def toJson(self):
        return json.dumps(self, default=lambda o: o.__dict__)

        
s = Job('John', 88 ,'sdad')
a=[]
a.append(s)
s1 = Job('John2', 88, 'adsa')
a.append(s1)
for s in a:
    print(s.description)

88
88


In [240]:
page_list = [10]

jobDic = {}
i=0
jobList = []

# jobs_categories = ['database+administrator','java+developer','database+analyst','database+architect','front+end+developer'
#                   ,'back+end+developer','software+developer','quality+assurance','mobile+developer','machine+learning','cloud+computing',
#                   '']

jobs_categories = ['java+developer']

with open('G:/Research/ScrapingDataFromIndeed.com/job_description.csv','a',encoding='utf-8',newline='') as f_output:
    csv_print = csv.writer(f_output)
    
    file_is_empty = os.stat('G:/Research/ScrapingDataFromIndeed.com/job_description.csv').st_size== 0
    if file_is_empty:
        csv_print.writerow(['Job_Title','Company','Location','Summary','Jobs_Description','Salary'])
    
    for job_list in jobs_categories:
        
        for page in page_list:
            source = requests.get('https://www.indeed.com/jobs?q='+job_list+'&start={}'.format(page)).text

            soup = BeautifulSoup(source,'lxml')
            #print(soup.prettify())

            #jobs = soup.find(class_='result')
            ##print(jobs.prettify())

            for jobs in soup.find_all(class_='result'):
                
                
                print("-------------------------------------------")
                requests.get('https://www.indeed.com/jobs?q='+job_list+'&start={}'.format(page)).text

                try:
                    
                    getJK = jobs.find(class_='title') 
                    getJK1 = getJK.find('a')
                    #print(getJK1)
                    getJK2 = re.search('"jl_(.+?)"',str(getJK1)).group(1)
                    #print(getJK2)
               
                    links = soup.find_all('script')
                    links2 = str(links)
                    word = re.search('tk (.*)',links2)
                    word2 =  str(word.groups())
                    match = re.search(r'["](.*?)["]',word2).group()
                    #print( match )

                    find_jk = soup.find(class_='result')
                    find_jk1 = find_jk.find('a')
                    find_jk2 = str(find_jk1)
                    find_jk3 = re.search(r'jk=(.*?)&',find_jk2).group(1)
                    #print(find_jk3)
                    
                    #source1 = requests.get('https://www.indeed.com/viewjob?'+find_jk3+''+match+'&from=serp&vjs=3').text

                    source1 = requests.get('https://www.indeed.com/viewjob?jk='+getJK2+'&'+match+'&from=serp&vjs=3').text


                    
                    soup1 = BeautifulSoup(source1,'lxml')
                    jobs_description = soup1.find(class_='jobsearch-jobDescriptionText').text.strip()
                    #print('*************************')
                    #print(jobs_description)
                    
                except Exception as e:
                    jobs_description = None

                try:
                    job_title = jobs.find(class_='title').text.strip()
                except Exception as e:
                    job_title = None
                #print('Job_Title' ,job_title)

                try:
                    company_name = jobs.find('span',class_='company').text.strip()
                except Exception as e:
                    company_name = None
                #print('Company_Name' ,company_name)

                try:
                    company_location = jobs.find('span',class_='location').text.strip()
                except Exception as e:
                    company_location = None
                #print('Company_Location' ,company_location)

                try:
                    summary = jobs.find(class_='summary').text.strip()
                except Exception as e:
                    summary = None
                #print('Summary' ,summary)

                try:
                    salary = jobs.find(class_='salary').text.strip()
                except Exception as e:
                    salary = None
                #print('Salary' ,salary)
                
                jobDic[i] = {'URI' : job_title ,'description' : company_location}
                i = i+1
                
                s = Job('https://www.indeed.com/viewjob?jk='+getJK2+'&'+match+'&from=serp&vjs=3', company_location , jobs_description)
                jobList.append(s)

                csv_print.writerow([job_title,company_name,company_location,summary,jobs_description,salary])

                time.sleep(2)
                
        #print(jobDic['description'])
                

-------------------------------------------
-------------------------------------------
-------------------------------------------
-------------------------------------------
-------------------------------------------
-------------------------------------------
-------------------------------------------
-------------------------------------------
-------------------------------------------
-------------------------------------------


In [174]:
for s in jobList:
    print(s.description)

United States
Waukegan, IL
Charleston, SC
Woodbridge, NJ 07095
Lake Mary, FL
Pittsburgh, PA
Union City, CA
Herndon, VA
Dallas, TX
United States


In [241]:
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from string import punctuation

nlp = spacy.blank('en')
stopwords = list(STOP_WORDS)
punctuation = punctuation +  '\n'
word_frequencies = {}
new_word_frequencies = []
finalJobDescriptionList = {}
jobListFinal = []

for s in jobList:
    word_frequencies = {}
    new_word_frequencies = []
    text = str(s.summary)
    URI =str(s.URI)
    doc = nlp(text)
    tokens = [token.text for token in doc]
    
    for word in doc:
             if word.text.lower() not in stopwords:
                if word.text.lower() not in punctuation:
                    if word.text.lower() not in word_frequencies.keys():
                        word_frequencies[word.text.lower()] = 1
                        new_word_frequencies.append(word.text.lower())
                    else:
                        word_frequencies[word.text.lower()] += 1
                        
    s = Job(URI,word_frequencies,new_word_frequencies)
    jobListFinal.append(s)
    print(new_word_frequencies)
    
for s in jobListFinal:
    print(s.URI)
    print(s.description)
    
# payload = {jobListFinal}
# s3 = Job('John2', 88)

# s3JSONData = json.dumps(s3.toJson())
# print(s3JSONData)

json_string = json.dumps([ob.__dict__ for ob in jobListFinal])
print(json_string)

payload = {'text1': json_string}

response = requests.get("http://localhost:8080/getJobs",params=payload)
print(response.status_code)
                

['position', 'overview', 'java', 'developer', 'engineer', '3', 'years', 'experience', 'time', 'benefits', 'onsite', 'd.c', 'reston', 'virginia', 'currently', 'remote', 'stay', 'home', 'orders', 'lifted', 'job', 'scope', 'responsibilities', 'serve', 'team', 'federal', 'government', 'agency', 'information', 'sharing', 'mission', 'responsible', 'building', 'applications', 'agile', 'development', 'methodology', 'dynamic', 'works', 'closely', 'client', 'ability', 'flexible', 'important', 'requirements', 'shift', 'looking', 'eager', 'learn', 'new', 'technologies', 'methodologies', 'atmosphere', 'congenial', 'welcomes', 'ideas', 'improve', 'productivity', 'simplify', 'code', 'ideal', 'candidate', 'enjoys', 'programming', 'challenges', 'minimal', 'direction', 'able', 'perform', 'web', 'application', 'design', 'coding', 'testing', 'debugging', 'enhancements', 'existing', 'software', 'conducted', 'spring', 'hibernate', 'role', 'include', 'limited', 'build', 'update', '8)', 'end', 'pages', 'html'

400


In [210]:
# NLP tokenizing the keywords

text = "Mid Level Java Developer Are you looking for a new and exciting job? What we are looking for: Excited and energized individuals who will be a good fit for the team. We will train you on the products and what you need to know. What you need to know: Be knowledgeable in JAVA and programming fundamentals. Pluses for experience/understanding web technologies, SEO, javascript, and Databases/security. Why Asponte?/Who We are? We are a small consulting firm and IBM premier business partner. We focus on selling services around the products we sell. We are a tight knit group, who like to have fun. We cater in lunches to the office, have snacks, have a few beers from time to time, and throw a football around to relax. Expected Duties: be trained in Portal and WCM, so that you will be a fully functional member of the team, doing java and WCM development. This can include everything from regular java applications, all the way to html based UI content. Travel can include up to 50%, although we prefer to schedule travel only as it makes sense, saving both our clients time and money To Apply, please use the form on our website to send a copy of your resume and cover letter. In your cover letter you should include the answer to this question: After exploring www.asponte.com[1] how do you see yourself fitting in with us? Any questions do not hesitate to ask 3-5 years java development experience ability to communicate in a professional environment authorization to work in the USA positive flexible attitude"

In [104]:
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from string import punctuation

nlp = spacy.blank('en')
stopwords = list(STOP_WORDS)
punctuation = punctuation +  '\n'
word_frequencies = {}
finalJobDescriptionList = {}

for i in range(len(jobDic)):
    text  = jobDic[i]['description']
    URI =  jobDic[i]['URI']
    doc = nlp(text)
    tokens = [token.text for token in doc]
    #print(tokens)

    for word in doc:
            if word.text.lower() not in stopwords:
                if word.text.lower() not in punctuation:
                    if word.text.lower() not in word_frequencies.keys():
                        word_frequencies[word.text.lower()] = 1
                    else:
                        word_frequencies[word.text.lower()] += 1
    
    finalJobDescriptionList[i] = {'URI' : URI ,'description' : word_frequencies}
                
print(finalJobDescriptionList)


{0: {'URI': 'Jr. Java Developer', 'description': {'charleston': 1, 'sc': 1, 'waukegan': 1, 'il': 1, 'united': 2, 'states': 2, 'woodbridge': 1, 'nj': 1, '07095': 1, 'north': 1, 'carolina': 1, 'lake': 1, 'mary': 1, 'fl': 1, 'pittsburgh': 1, 'pa': 1, 'new': 1, 'york': 1, 'ny': 1, '10038': 1, 'financial': 1, 'district': 1, 'area': 1, 'union': 1, 'city': 1}}, 1: {'URI': 'AS/400 RPG DEVELOPER + JAVA, # 10679', 'description': {'charleston': 1, 'sc': 1, 'waukegan': 1, 'il': 1, 'united': 2, 'states': 2, 'woodbridge': 1, 'nj': 1, '07095': 1, 'north': 1, 'carolina': 1, 'lake': 1, 'mary': 1, 'fl': 1, 'pittsburgh': 1, 'pa': 1, 'new': 1, 'york': 1, 'ny': 1, '10038': 1, 'financial': 1, 'district': 1, 'area': 1, 'union': 1, 'city': 1}}, 2: {'URI': 'Java Developer', 'description': {'charleston': 1, 'sc': 1, 'waukegan': 1, 'il': 1, 'united': 2, 'states': 2, 'woodbridge': 1, 'nj': 1, '07095': 1, 'north': 1, 'carolina': 1, 'lake': 1, 'mary': 1, 'fl': 1, 'pittsburgh': 1, 'pa': 1, 'new': 1, 'york': 1, 'ny':

In [118]:
# ApI request to JAVA
#text1="testing"
import json

json_format = json.dumps(finalJobDescriptionList)
print(json_format)

payload = {'text1': json_format}
response = requests.get("http://localhost:8080/getJobDescription" ,params=payload)
print(response.status_code)

{"0": {"URI": "Jr. Java Developer", "description": {"charleston": 1, "sc": 1, "waukegan": 1, "il": 1, "united": 2, "states": 2, "woodbridge": 1, "nj": 1, "07095": 1, "north": 1, "carolina": 1, "lake": 1, "mary": 1, "fl": 1, "pittsburgh": 1, "pa": 1, "new": 1, "york": 1, "ny": 1, "10038": 1, "financial": 1, "district": 1, "area": 1, "union": 1, "city": 1}}, "1": {"URI": "AS/400 RPG DEVELOPER + JAVA, # 10679", "description": {"charleston": 1, "sc": 1, "waukegan": 1, "il": 1, "united": 2, "states": 2, "woodbridge": 1, "nj": 1, "07095": 1, "north": 1, "carolina": 1, "lake": 1, "mary": 1, "fl": 1, "pittsburgh": 1, "pa": 1, "new": 1, "york": 1, "ny": 1, "10038": 1, "financial": 1, "district": 1, "area": 1, "union": 1, "city": 1}}, "2": {"URI": "Java Developer", "description": {"charleston": 1, "sc": 1, "waukegan": 1, "il": 1, "united": 2, "states": 2, "woodbridge": 1, "nj": 1, "07095": 1, "north": 1, "carolina": 1, "lake": 1, "mary": 1, "fl": 1, "pittsburgh": 1, "pa": 1, "new": 1, "york": 1,

In [68]:
#Loading Resume and  Extracting Data
import pickle
import random

In [69]:
train_data = pickle.load(open('train_data.pkl','rb'))
train_data[0]

('Govardhana K Senior Software Engineer  Bengaluru, Karnataka, Karnataka - Email me on Indeed: indeed.com/r/Govardhana-K/ b2de315d95905b68  Total IT experience 5 Years 6 Months Cloud Lending Solutions INC 4 Month • Salesforce Developer Oracle 5 Years 2 Month • Core Java Developer Languages Core Java, Go Lang Oracle PL-SQL programming, Sales Force Developer with APEX.  Designations & Promotions  Willing to relocate: Anywhere  WORK EXPERIENCE  Senior Software Engineer  Cloud Lending Solutions -  Bangalore, Karnataka -  January 2018 to Present  Present  Senior Consultant  Oracle -  Bangalore, Karnataka -  November 2016 to December 2017  Staff Consultant  Oracle -  Bangalore, Karnataka -  January 2014 to October 2016  Associate Consultant  Oracle -  Bangalore, Karnataka -  November 2012 to December 2013  EDUCATION  B.E in Computer Science Engineering  Adithya Institute of Technology -  Tamil Nadu  September 2008 to June 2012  https://www.indeed.com/r/Govardhana-K/b2de315d95905b68?isid=rex-

In [70]:
def train_model(train_data):
    if 'ner' not in nlp.pipe_names:
        ner = nlp.create_pipe('ner')
        nlp.add_pipe(ner, last = True)
        
    for _, annotation in train_data:
        for ent in annotation['entities']:
            ner.add_label(ent[2])
            
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
    with nlp.disable_pipes(*other_pipes):
        optimizer = nlp.begin_training()
        for itn in range(10):
            print("starting iteration " + str(itn))
            random.shuffle(train_data)
            losses = {}
            index = 0
            for text, annotation in train_data:
                try:
                    nlp.update(
                        [text],
                        [annotation],
                        drop=0.2,
                        sgd=optimizer,
                        losses=losses)
                except Exception as e:
                    pass
                    
                        
            print(losses)  

In [71]:
train_model(train_data)

starting iteration 0
{'ner': 13854.666263108866}
starting iteration 1
{'ner': 7708.9631790547355}
starting iteration 2
{'ner': 8479.110717201578}
starting iteration 3
{'ner': 5873.730081265468}
starting iteration 4
{'ner': 7525.433863721018}
starting iteration 5
{'ner': 5333.9183280053885}
starting iteration 6
{'ner': 5619.327874768101}
starting iteration 7
{'ner': 5029.440601398649}
starting iteration 8
{'ner': 5455.853326536679}
starting iteration 9
{'ner': 5554.030947513798}


In [72]:
nlp.to_disk('nlp_model')
nlp_model = spacy.load('nlp_model')
train_data[0][0]
doc = nlp_model(train_data[0][0])
for ent in doc.ents:
    print(f'{ent.label_.upper():{30}}-{ent.text}')

NAME                          -Sumit Kubade
LOCATION                      -SAP -
COMPANIES WORKED AT           -SAP FI
LOCATION                      -Pune
DESIGNATION                   -SAP FINANCE MODULE
LOCATION                      -Pune
DESIGNATION                   -SAP - FI Support Consultant
COMPANIES WORKED AT           -SAP FI
COLLEGE NAME                  -Indian Institute Of Company Secretory Of India (Appeared)
GRADUATION YEAR               -2014
DEGREE                        -B.com
COLLEGE NAME                  -Pune University
SKILLS                        -SAP (1 year)


In [73]:
!pip install PyMuPDF

You should consider upgrading via the 'c:\users\akalanka\anaconda3\python.exe -m pip install --upgrade pip' command.


In [74]:
import sys,fitz
fname = 'Test3.pdf'
doc = fitz.open(fname)
text = ""
for page in doc:
    text = text + str(page.getText())

tx = " ".join(text.split('\n'))
print(tx)  

Java Developer  Hyderabad, Telangana  To work in a firm with a professional work driven environment where I can utilize and apply   my knowledge, skills which would enable me as a fresh graduate to grow while fulfilling   organizational goals.      BASIC ACADEMIC CREDENTIALS      #readytowork  Willing to relocate: Anywhere  Work Experience  Java Developer  Kansas IT Solutions - Hyderabad, Telangana  June 2019 to July 2019  In an internship program I have successfully completed internship on Java Developer from  Kansas IT Solutions , Hyderabad .The duration of the project was from 3rd June to 3rd July .  Education  Bachelor's in Computer Science Engineering  Osmania University - Hyderabad, Telangana  June 2016 to June 2020  Higher Secondary(12th Pass) in Mathematics,Physics,Chemistry  Telangana Board of Secondary Education - Hyderabad, Telangana  June 2014 to June 2016    SKILLS  ● Python  ● Java  ● C  ● Web Development  ● Data Structure  ● Dbms  ● Html5  ● Programming  Projects / Paper

In [76]:
doc = nlp_model(tx)
for ent in doc.ents:
    print(f'{ent.label_.upper():{30}}-{ent.text}')

NAME                          -Java Developer
LOCATION                      -Hyderabad
DESIGNATION                   -Java Developer
DEGREE                        -Bachelor's in Computer Science Engineering
COLLEGE NAME                  -Osmania University


In [77]:
docCV = nlp(tx)
word_frequenciesCV = {}
for word in docCV:
    if word.text.lower() not in stopwords:
        if word.text.lower() not in punctuation:
            if word.text.lower() not in word_frequenciesCV.keys():
                word_frequenciesCV[word.text.lower()] = 1
            else:
                word_frequenciesCV[word.text.lower()] += 1
            
print(word_frequenciesCV)

{'java': 4, 'developer': 3, ' ': 29, 'hyderabad': 5, 'telangana': 5, 'work': 3, 'firm': 1, 'professional': 1, 'driven': 1, 'environment': 1, 'utilize': 1, 'apply': 1, '  ': 2, 'knowledge': 1, 'skills': 2, 'enable': 1, 'fresh': 1, 'graduate': 1, 'grow': 1, 'fulfilling': 1, 'organizational': 1, 'goals': 1, '     ': 2, 'basic': 1, 'academic': 1, 'credentials': 1, 'readytowork': 1, 'willing': 1, 'relocate': 1, 'experience': 1, 'kansas': 2, 'solutions': 2, 'june': 6, '2019': 2, 'july': 2, 'internship': 2, 'program': 1, 'successfully': 1, 'completed': 1, '.the': 1, 'duration': 1, 'project': 3, '3rd': 2, 'education': 2, 'bachelor': 1, 'computer': 1, 'science': 1, 'engineering': 1, 'osmania': 1, 'university': 1, '2016': 2, '2020': 2, 'higher': 1, 'secondary(12th': 1, 'pass': 1, 'mathematics': 1, 'physics': 1, 'chemistry': 1, 'board': 1, 'secondary': 1, '2014': 1, '   ': 2, '●': 8, 'python': 2, 'c': 1, 'web': 1, 'development': 1, 'data': 1, 'structure': 1, 'dbms': 1, 'html5': 1, 'programming': 

In [78]:
payload = {'text1': word_frequenciesCV}
response = requests.get("http://localhost:8080/getJobDescription" ,params=payload)
print(response.status_code)

200


In [130]:
class Student:
    def __init__(self, name, age, major):
        self.name = name
        self.age = age
        self.major = major

s = Student('John', 88, None)
a=[]
a.append(s)
s1 = Student('John2', 88, None)
a.append(s1)
for s in a:
    print(s.name)

John
John2
